In [5]:
import matplotlib.pyplot
import sys

In [52]:
def input() -> ([], []) :
    with open("filter.txt", "r") as filter:
        filter_buf = filter.read()
    
    with open("image.txt", "r") as image:
        image_buf = image.read()

    filter = [0]*(3*3)
    image = [0]*(64*64) # row major formate of stroing an image 

    filter = [int(v) for v in filter_buf.split('\n')]
    
    i = 0
    for v in image_buf.split('\n'):
        try:
            image[i] = int(v)
        except:
            continue

        i += 1

    return filter, image

In [139]:
class util:
    def all_vals(self, image, i)-> (int, int, int, int, int, int, int, int):
        a0 = image[i+0-64]
        a1 = image[i+1-64]
        a2 = image[i+2-64]
        a3 = image[i+63-64]
        a4 = image[i+64-64]
        a5 = image[i+65-64]
        a6 = image[i+126-64]
        a7 = image[i+127-64]
        a8 = image[i+128-64]
        return a0, a1, a2, a3, a4, a5, a6, a7, a8
    
    def bin_to_int(self, val) -> int:
        res = 0
        i = 0
        while ( val > 0 ):
            n = val%10
            if (n == 1):
                res += 2**i
            val //= 10
            i += 1
        
        return res
    
    def int_to_bin(self, val) -> int:
        res = 0
        i = 0
        while (val > 0):
            n = val%2
            if (n == 1):
                res += 10**i
            val //= 2
            i +=  1

        return res

In [143]:
def MAC(filter, image) -> ([], int, int):
    f0 = filter[0]
    f1 = filter[1]
    f2 = filter[2]
    f3 = filter[3]
    f4 = filter[4]
    f5 = filter[5]
    f6 = filter[6]
    f7 = filter[7]
    f8 = filter[8]

    i = 0
    output_image = [0]*(64*64)
    max_pix = -2**32
    min_pix = 2**32
    while True:
        """
            a0 a1 a2            a0 a1 a2
            a3 a4 a5            a3 a4 a5
            a6 a7 a8            a6 a7 a8

        """
        if i == 4096:
            break
        utils = util()
        if (i == 0):
            a0 = a1 = a2 = 0
            a0 = a3 = a6 = 0
            a4 = image[i+64-64]
            a5 = image[i+65-64]
            a7 = image[i+127-64]
            a8 = image[i+128-64]
        elif (i == 63):
            a0 = a1 = a2 = 0
            a5 = a6 = 0
            a3 = image[i+63-64]
            a4 = image[i+64-64]
            a7 = image[i+127-64]
            a8 = image[i+128-64]
        elif (i == 4096-64):
            a0 = a3 = a6 = 0
            a7 = a8 = 0
            a1 = image[i+1-64]
            a2 = image[i+2-64]
            a4 = image[i+64-64]
            a5 = image[i+65-64]
        elif (i == 4096 - 1):
            a2 = a5 = a8 = a6 = a7 = 0
            a0 = image[i+0-64]
            a1 = image[i+1-64]
            a3 = image[i+63-64]
            a4 = image[i+64-64]
        elif (i < 64):
            a0 = a1 = a2 = 0
            a3 = image[i+63-64]
            a4 = image[i+64-64]
            a5 = image[i+65-64]
            a6 = image[i+126-64]
            a7 = image[i+127-64]
            a8 = image[i+128-64]
        elif (i%64 == 0):
            a0 = a3 = a6 = 0
            a1 = image[i+1-64]
            a2 = image[i+2-64]
            a4 = image[i+64-64]
            a5 = image[i+65-64]
            a7 = image[i+127-64]
            a8 = image[i+128-64]
        elif (i >= 4032):
            a6 = a7 = a8 = 0
            a0 = image[i+0-64]
            a1 = image[i+1-64]
            a2 = image[i+2-64]
            a3 = image[i+63-64]
            a4 = image[i+64-64]
            a5 = image[i+65-64]
        elif (i%63 == 0):
            a2 = a5 = a8 = 0
            a0 = image[i+0-64]
            a1 = image[i+1-64]
            a3 = image[i+63-64]
            a4 = image[i+64-64]
            a6 = image[i+126-64]
            a7 = image[i+127-64]
        else:
            a0, a1, a2, a3, a4, a5, a6, a7, a8 = utils.all_vals(image, i)
        
        # correction, f_i are unsigned, a_i are signed, output_image would be unsigned
        temp = (utils.bin_to_int(f0)*utils.bin_to_int(a0) + utils.bin_to_int(f1)*utils.bin_to_int(a1) + utils.bin_to_int(f2)*utils.bin_to_int(a2) + utils.bin_to_int(f3)*utils.bin_to_int(a3) + utils.bin_to_int(f4)*utils.bin_to_int(a4) + utils.bin_to_int(f5)*utils.bin_to_int(a5) + utils.bin_to_int(f6)*utils.bin_to_int(a6) + utils.bin_to_int(f7)*utils.bin_to_int(a7) + utils.bin_to_int(f8)*utils.bin_to_int(a8))
        
        output_image[i] = utils.int_to_bin(temp)
        if (temp < min_pix):
            min_pix = temp
        if ( temp > max_pix ):
            max_pix = temp 

        i += 1
    
    return output_image, min_pix, max_pix


In [197]:
def FSM(output, min_pix, max_pix):
    utils = util()
    res = [0]*(64*64)
    # also use the unsinged version here
    for i in range(len(output)):
        res[i] = utils.int_to_bin((utils.bin_to_int(output[i]) - (min_pix))*(255/((max_pix) - (min_pix))))
    
    return res

In [198]:
def debug():
    utils = util()
    res = utils.bin_to_int(11)
    return res


In [199]:
def print_foo(output, min_pix, max_pix):
    print("The output array of un_normalised, and taken as signed bits that is to be corrected is:")
    print(f"\t{output}")
    print(f"\tThe minimum value of pixel for normalising is:- {min_pix}")
    print(f"\tThe maximum vlaue of pixel for normalising is:- {max_pix}")

In [200]:
def main():
    filter_rm, image_rm = input()
    output, min_pix, max_pix = MAC(filter_rm, image_rm)
    norm_output = FSM(output, min_pix, max_pix)

    print_foo(norm_output, min_pix, max_pix)

    # print(debug())
    

if __name__ == "__main__":
    main()

The output array of un_normalised, and taken as signed bits that is to be corrected is:
	[1000100, 10100100, 10100100, 10100110, 10100110, 10100110, 10100110, 10101000, 10010010, 10010010, 1101010, 1010000, 1000110, 110110, 1000010, 110110, 110010, 1010, 101000, 101100, 100110, 11100, 100110, 101000, 101110, 110010, 111010, 110010, 10100, 101110, 1000100, 1111000, 1110110, 1101100, 1100100, 1000110, 111010, 1010010, 1010110, 10000000, 10010110, 10100000, 10100100, 10100000, 10100000, 10010110, 10000110, 1101000, 1010110, 100100, 100010, 101000, 101010, 1000000, 111100, 1011000, 100000, 110, 1100, 100, 10100, 11110, 11110, 11100, 1111100, 11100000, 11100000, 11100000, 11100000, 11100010, 11100010, 11100100, 11000110, 10111010, 10011110, 1110010, 1100000, 1011010, 1011010, 110010, 111000, 101100, 101110, 110010, 101110, 101100, 101010, 111000, 1000100, 1001010, 110010, 101100, 1000000, 10001010, 10100000, 10111100, 10111000, 10100000, 10010010, 10000010, 1110100, 10100010, 10110110, 1100